# Portainer使用
　　参考：[]()<br>
　　　　　[docker Portainer 文档](https://blog.csdn.net/jiangyucaonima/article/details/95203644)<br>
　　　　　[Docker管理面板系列——Portainer(简单、美观、强大)](http://www.senra.me/docker-management-panel-series-portainer/)<br>
　　　　　[]()<br>
　　　　　[使用Docker安装Portainer](https://www.cnblogs.com/nsky/p/13368721.html)<br>
　　　　　[docker开启认证的远程端口2376配置教程](https://blog.csdn.net/qq_21187515/article/details/90262324)<br>
　　　　　[Portainer -- Docker可视化管理工具的安装配置及使用](https://blog.csdn.net/A632189007/article/details/78779920)<br>
## 一、Portainer介绍
　　Docker 图形化管理提供了很多工具，有Portainer、Docker UI、Shipyard等等，本文主要介绍Portainer。<br>
　　Portainer是Docker的图形化管理工具，提供状态显示面板、应用模板快速部署、容器镜像网络数据卷的基本操作（包括上传下载镜像，创建容器等操作）、事件日志显示、容器控制台操作、Swarm集群和服务等集中管理和操作、登录用户管理和控制等功能。功能十分全面，基本能满足中小型单位对容器管理的全部需求。<br>
　　Portainer是一个轻量级的Docker环境管理UI，可以用来管理Docker宿主机和Docker Swarm集群。<br>
　　Portainer主要功能：<br>
　　　　1、提供状态显示面板：显示主机或者Swarm集群上有多少镜像，容器、网络、数据卷等；<br>
　　　　2、应用模板快速部署：可以使用预存的模板或者自己定制的模板快速部署；<br>
　　　　3、镜像网络数据卷管理：通过页面进行管理和操作，例如构建镜像，增删数据卷等；<br>
　　　　4、事件日志显示：对任何操作有记录，并且有页面可以显示审计日志；<br>
　　　　5、容器控制台操作：查看容器，启停容器，查看容器占用的性能（内存，CPU等）<br>
　　　　6、Swarm集群管理：可以管理Swarm集群，是最大的优点；<br>
　　　　7、登录用户管理：有完备的用户系统，权限控制（高级功能需付费）。<br>
## 二、Portainer安装配置
　　建议使用docker安装Portainer.<br>
### 2.1 下载Portainer镜像
>~\\$ docker search portainer　　# 查询当前有哪些Portainer镜像<br>
~\\$ docker pull portainer/portainer　　　# 下载镜像<br>

### 2.2 创建数据卷
　　使用docker volume创建容器外列，create后是外列的名称，即外列的名称是portainer_data。<br>
　　主要作用是创建一个空间用来存储已经创建好的容器内的数据，而这个空间和这个容器的关联键就是外列的名称，这样做的好处是如果容器挂掉，可以使用空间内的容器的备份数据保证容器正常运行。<br>
>~\\# docker volume create portainer_data

### 2.3 容器开通2375端口
　　当使用Docker集群时，所有的slave丛机需开启2375端口。<br>
#### 2.3.1 Ubuntu
　　未验证<br>
sudo vim /etc/default/docker<br>
DOCKER_OPTS="-H=unix:///var/run/docker.sock -H=0.0.0.0:2375"<br>

#### 2.3.2 Centos
　　未验证<br>
sudo vim /etc/sysconfig/docker
  other_args='-H tcp://0.0.0.0:2375 -H unix:///var/run/docker.sock'　#centos6下使用<br>
  OPTIONS='-H tcp://0.0.0.0:2375 -H unix:///var/run/docker.sock'　　　#centos7下使用<br>

#### 2.3.3
　　验证有效<br>
>~\# cp  /lib/systemd/system/docker.service /lib/systemd/system/docker.service.bak　　 # 先做备份<br>
~\# vim /lib/systemd/system/docker.service <br>
~\# \# 找到ExecStart行改成这样的：  ExecStart=/usr/bin/dockerd -H tcp://0.0.0.0:2375 -H unix:///var/run/docker.sock<br>
~\# systemctl daemon-reload<br>
~\# systemctl restart docker　　　　\# 重启docker服务<br>

#### 2.3.4 开放防火墙端口2375
　　由于防火墙的原因，需要打开2375端口防火墙，才能远程访问，可以在防火墙中开放该端口，也可以直接关闭防火墙。<br> 
　　直接关闭防火墙：<br>
>systemctl status firewalld.service　　　# 查看防火墙状态，绿的running表示防火墙开启<br> 
systemctl stop firewalld.service　　　　# 执行关闭命令 <br> 
systemctl status firewalld.service　　　# 再次执行查看防火墙命令 <br> 
systemctl disable firewalld.service　　　# 执行开机禁用防火墙自启命令 <br> 

　　开放端口：<br>
>~\# firewall-cmd --zone=public --add-port=2375/tcp --permanent   <br> 
~\# firewall-cmd --reload   <br>
~\# firewall-cmd --zone=public --query-port=2375/tcp　　# 验证2375端口打开是否生效，看到yes及表示生效<br>

#### 2.3.5 查看端口号使用
>~\# netstat -plnt  　　\# 查看端口号使用 <br>
　　tcp 　　0　　0 0.0.0.0:50008　　　 0.0.0.0:*　　　LISTEN　　　26500/sshd<br>
　　tcp6　　0　　0 :::2375　　　 　　　 :::*　　　　　LISTEN 　　　25595/dockerd<br> 


## 三、运行Portainer
　　Portainer运行方式两种: 本地模式(单机)、远程模式(集群)<br>
### 3.1 本地运行Portainer
　　如果仅有一个docker宿主机，则可使用单机模式运行Portainer，十分简单，只需要一条语句即可启动容器，来管理本地机器上的docker镜像、容器等数据。<br>
>~\\$ \# 本机模式<br>
docker run -d -p 9000:9000 --name portainer \\<br>
　　-v /var/run/docker.sock:/var/run/docker.sock \\<br>
　　-v portainer_data:/data \\<br>
　　--restart=always \\<br>
　　portainer/portainer<br>

　　参数说明：<br>

|参数|说明|
|:---|:---|
|-d|容器在后台运行|
|-p 9000:9000|宿主机9000端口映射容器中的9000端口|
|--name portainer|容器命名为portainer|
|-v portainer_data:/data|数据持久化，把宿主机portainer_data数据卷挂载到容器/data目录|
|-v /var/run/docker.sock:/var/run/docker.sock|把宿主机的Docker守护进程(Docker daemon)默认监听的Unix域套接字挂载到容器中|


　　说明：
　　默认情况下，Portainer将其数据存储在/data（Linux）或C:\\data（Windows）目录下的容器中。在重新启动/升级Portainer容器后，您需要保存Portainer数据以保留更改。您可以使用绑定装载来保存Docker主机文件夹上的数据（-v portainer_data:/data）。<br>

　　查看容器运行：docker ps -l<br>
　　访问Portainer容器：http://主机ip:9000<br>
　　选择Local，代表本地模式，portainer仅管理本机的docker容器<br>
### 3.2 集群运行Portainer
　　更多的情况下，我们会有一个docker集群，可能有几台机器，也可能有几十台机器，因此，进行集群管理就十分重要了，Portainer也支持集群管理，Portainer可以和Swarm一起来进行集群管理操作。<br>
>~\\$ \# 远程模式（mater主节点）<br>
~\\$ docker run -d -p 9000:9000 --restart=always --name prtainer portainer/portainer<br><br>

#### 3.2.1 访问Portainer容器
　　访问地址：http://IP:9000<br>
　　首次登录需要设置admin的密码<br>
　　选择Remote，代表远程模式，名称随意，在Endpoint URL中填写docker节点的地址:docker远程端口（docker安装教程中的设置的-H 0.0.0.0:2375中的2375）；同样点击左边栏的”Endpoints” -> “+ add endpoint”,通过Docker API添加其他集群slave节点:ip:2375；添加之后,点击左边栏的”Home”, 右边就可以看到节点信息了,可以进行切换操作。<br>

#### 3.2.2 Agent方式管理多台主机
　　由于Docker API方式需要重启Docker服务，会影响现有应用运行，可以采用Agent 方式管理。<br>
**管理节点部署Master：**<br>
>~\\$ docker run -d -p 9000:9000 --name portainer --restart=always \\<br>
　　-v /var/run/docker.sock:/var/run/docker.sock \\<br>
　　-v  portainer_data:/data \\<br>
　　portainer/portainer <br>

**节点主机部署Agent：**<br>
>~\\$ docker run -d -p 9001:9001 --name portainer --restart=always \\<br>
　　-v /var/run/docker.sock:/var/run/docker.sock \\<br>
　　-v /var/lib/docker/volumes:/var/lib/docker/volumes \\<br>
　　portainer/agent<br>

　　在Portainer管理页面添加Agent主机。<br>




docker Portainer 文档